# Lab 5 Data Structures

Resources
- https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.io.json.json_normalize.html
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html
- https://docs.python.org/2/library/xml.etree.elementtree.html

In [23]:
# import modules
import pandas as pd
import numpy as np

from pandas.io.json import json_normalize #special package in pandas
import json
from lxml import etree
import sqlite3

from google.colab import drive
drive.mount('/content/drive', force_remount = False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Part A: Reading json

In [2]:
# Run this cell to import the following JSON strings
players = '[{"name":"Gazinsky","team":"Russia"},{"name":"Dzyuba","team":"Russia"},{"name":"Lukaku","team":"Belgium"}]'
stadium_data = '{"stadiums":[{"name": "Ekaterinburg Arena","city": "Ekaterinburg"},{"name": "Luzhniki Stadium","city": "Moscow"},{"name": "Nizhny Novgorod Stadium","city": "Nizhny Novgorod"}]}'

In [5]:
# load players string as a json object (using json.loads)
pjson = json.loads(players)

# print the dictionary containing information about the third player only
# HINT: Once the data is in python data structures (i.e. lists/dictionaries) 
print(pjson[2]) # Zero indexed -- Player #3 is at index 2
print("-----------")

# navigate through the nested lists and dictionaries to extract the relevant information
# HINT: Remember, the index starts at 0, so the first player is index 0, the second player is index 1, etc.
for player in pjson:
  print(player)

{'name': 'Lukaku', 'team': 'Belgium'}
-----------
{'name': 'Gazinsky', 'team': 'Russia'}
{'name': 'Dzyuba', 'team': 'Russia'}
{'name': 'Lukaku', 'team': 'Belgium'}


In [11]:
# load stadium_data as a json object
sjson = json.loads(stadium_data)

# print the city of Luzhniki Stadium by navigating through the nested dictionaries and lists (key="city")
print(sjson["stadiums"][1]['city'])

Moscow


In [27]:
# read world cup match data from the worldcup.json file into a json object
with open("/content/drive/MyDrive/Colab Notebooks/INST 447/Module 5/worldcup.json") as f:
  mjson = json.load(f)

# print the date of the first match (key="date")
# HINT: Once the data is in python data structures (i.e. lists/dictionaries)
print(mjson['rounds'][0]['matches'][0]['date'])
print("------------")

# navigate through the nested lists and dictionaries to extract the relevant information
for round in mjson['rounds']:
  print(round['name'] + ": " + str(len(round['matches'])) + " matches")

2018-06-14
------------
Matchday 1: 1 matches
Matchday 2: 3 matches
Matchday 3: 4 matches
Matchday 4: 3 matches
Matchday 5: 3 matches
Matchday 6: 3 matches
Matchday 7: 3 matches
Matchday 8: 3 matches
Matchday 9: 3 matches
Matchday 10: 3 matches
Matchday 11: 3 matches
Matchday 12: 4 matches
Matchday 13: 4 matches
Matchday 14: 4 matches
Matchday 15: 4 matches
Round of 16: 8 matches
Quarter-finals: 4 matches
Semi-finals: 2 matches
Match for third place: 1 matches
Final: 1 matches


In [32]:
# read world cup team data from the worldcup.teams.json file into json object
with open("/content/drive/MyDrive/Colab Notebooks/INST 447/Module 5/worldcup.teams.json") as f:
  wctjson = json.load(f)

# print the name of the first team in the team data set (key="name")
# HINT: Once the data is in python data structures (i.e. lists/dictionaries)
print(wctjson['teams'][0]['name'])
print('--------------------')

# navigate through the nested lists and dictionaries to extract the relevant information
for team in wctjson['teams']:
  print(team['name'] + " (" + team['code'] + ")")

Egypt
--------------------
Egypt (EGY)
Morocco (MAR)
Tunisia (TUN)
Senegal (SEN)
Nigeria (NGA)
Japan (JPN)
South Korea (KOR)
Costa Rica (CRC)
Panama (PAN)
Belgium (BEL)
Germany (GER)
Spain (ESP)
France (FRA)
Portugal (POR)
Denmark (DEN)
Poland (POL)
Sweden (SWE)
Switzerland (SUI)
Croatia (CRO)
Serbia (SRB)
Russia (RUS)
Iceland (ISL)
England (ENG)
Iran (IRN)
Saudi Arabia (KSA)
Mexico (MEX)
Australia (AUS)
Argentina (ARG)
Brazil (BRA)
Uruguay (URU)
Colombia (COL)
Peru (PER)


In [33]:
# manually create a list of dictionaries with the following soccer associations and continents as the keys & values
# The keys should be  "continent" and "association". 

# Europe - Union of European Football Associations 
# Asia - Asian Football Confederation
# Africa - Confederation Africaine de Football 
socc_dict = [
  {
      "continent": "Europe",
      "association": "Union of European Football Associations"
  },           
  {
      "continent": "Asia",
      "association": "Asian Football Confederation"
  },  
  {
      "continent": "Africa",
      "association": "Confederation Africaine de Football"
  },  
]

# Part B: Flattening json

In [34]:
# flatten the players json object created in Part A (second cell) into a data frame with one row per player
player_df = json_normalize(pjson)

# print first few rows of data set
player_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,team
0,Gazinsky,Russia
1,Dzyuba,Russia
2,Lukaku,Belgium


In [39]:
# flatten the stadium data json object created in Part A (third cell) into a data frame with one row per stadium
stadium_df = json_normalize(sjson, record_path = 'stadiums')

# print first few rows of data set
stadium_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,city
0,Ekaterinburg Arena,Ekaterinburg
1,Luzhniki Stadium,Moscow
2,Nizhny Novgorod Stadium,Nizhny Novgorod


In [46]:
# flatten the world cup match json object created in Part A (fourth cell)  into a data frame with one row per match
# HINT: The data you want is nested in "rounds", and further in the "matches" key in the world cup match json. You are going to have to get rid of two outer layers.
# HINT: You should include both rounds & matches as nested keys in record path
# (see https://stackoverflow.com/questions/47242845/pandas-io-json-json-normalize-with-very-nested-json)
# e.g. record_path=["level1key",["level2key"]]
# where level1key = "rounds" and level2key = "matches"
match_df = json_normalize(mjson, record_path = ['rounds', ['matches']])

# print number of rows
print("Rows " + str(match_df.shape[0]))

# print first few rows of data set
match_df.head()

Rows 64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


,num,date,time,score1,score2,score1i,score2i,goals1,goals2,group,city,timezone,team1.name,team1.code,team2.name,team2.code,stadium.key,stadium.name,score1et,score2et,score1p,score2p,knockout
0,1,2018-06-14,18:00,5,0,2,0,"[{'name': 'Gazinsky', 'minute': 12, 'score1': ...",[],Group A,Moscow,UTC+3,Russia,RUS,Saudi Arabia,KSA,luzhniki,Luzhniki Stadium,NaN,NaN,NaN,NaN,NaN
1,2,2018-06-15,17:00,0,1,0,0,[],"[{'name': 'Giménez', 'minute': 89, 'score1': 0...",Group A,Ekaterinburg,UTC+5,Egypt,EGY,Uruguay,URU,ekaterinburg,Ekaterinburg Arena,NaN,NaN,NaN,NaN,NaN
2,3,2018-06-15,21:00,3,3,2,1,"[{'name': 'Ronaldo', 'minute': 4, 'score1': 1,...","[{'name': 'Costa', 'minute': 24, 'score1': 1, ...",Group B,Sochi,UTC+3,Portugal,POR,Spain,ESP,fisht,Fisht Stadium,NaN,NaN,NaN,NaN,NaN
3,4,2018-06-15,18:00,0,1,0,0,[],"[{'name': 'Bouhaddouz', 'minute': 90, 'offset'...",Group B,Saint Petersburg,UTC+3,Morocco,MAR,Iran,IRN,saintpetersburg,Saint Petersburg Stadium,NaN,NaN,NaN,NaN,NaN
4,5,2018-06-16,13:00,2,1,0,0,"[{'name': 'Griezmann', 'minute': 58, 'score1':...","[{'name': 'Jedinak', 'minute': 62, 'score1': 1...",Group C,Kazan,UTC+3,France,FRA,Australia,AUS,kazan,Kazan Arena,NaN,NaN,NaN,NaN,NaN


In [48]:
# flatten the team data json object created in Part A (fifth cell) into a data frame with one row per team
wct_df = json_normalize(wctjson, record_path = 'teams')

# print number of rows
print("Rows " + str(wct_df.shape[0]))

# print first few rows of data set
wct_df.head()

Rows 32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,code,continent,assoc.key,assoc.name,assoc.continental.name,assoc.continental.code
0,Egypt,EGY,Africa,egy,Egyptian Football Association,Confédération Africaine de Football (CAF),CAF
1,Morocco,MAR,Africa,mar,Fédération Royale Marocaine de Football,Confédération Africaine de Football (CAF),CAF
2,Tunisia,TUN,Africa,tun,Fédération Tunisienne de Football,Confédération Africaine de Football (CAF),CAF
3,Senegal,SEN,Africa,sen,Fédération Sénégalaise de Football,Confédération Africaine de Football (CAF),CAF
4,Nigeria,NGA,Africa,nga,Nigeria Football Federation,Confédération Africaine de Football (CAF),CAF


In [50]:
# flatten the soccer association list of dictionaries created in Part A (sixth cell) into a data frame with one row per association
socc_df = json_normalize(socc_dict)

# print number of rows
print(socc_df.shape[0])

# print first few rows of data set
socc_df.head()

3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,continent,association
0,Europe,Union of European Football Associations
1,Asia,Asian Football Confederation
2,Africa,Confederation Africaine de Football


# Part C: SQL

In [52]:
# create a database and sql connection 
con = sqlite3.connect("/content/drive/MyDrive/Colab Notebooks/INST 447/Module 5/database.sqlite")

In [53]:
# save the players data set created in part B as a sql table in your database
player_df.to_sql('players', con, if_exists = 'replace')

In [54]:
# save the stadiums data set created in part B as a sql table in your database
stadium_df.to_sql('stadiums', con, if_exists = 'replace')

In [58]:
# save the world cup match data set created in part B as a sql table in your database
# (you'll need to FIRST slice the data frame by selecting only the columns we want. Drop goals1 & goals2)
match_df[match_df.columns.drop(['goals1', 'goals2'])].to_sql('matches', con, if_exists = 'replace')

In [65]:
# save the teams data set created in part B as a sql table in your database. Keep the following
# variables: "code","continent", & "name"
wct_df[['code', 'continent', 'name']].to_sql('teams', con, if_exists = 'replace')

In [70]:
# From the world cup match data table, use SQL syntax to select only those rows where Mexico played as team1 (team1.name) OR team2 (team2.name) from the database, and save these rows as a data frame
# HINT: You'll need to use WHERE to select the right rows
# VERY IMPORTANT HINT: if a column name includes a period (.), you must enclose the
# column name  AND the value for the WHERE statement in double quotes, and enclose the entire query in single quotes
# eg. 'SELECT * FROM df WHERE "col1.name"="Atlanta" OR "col2.name"="Atlanta"'
df1 = pd.read_sql_query('SELECT * FROM matches WHERE "team1.name" = "Mexico" OR "team2.name" = "Mexico"', con)

# count the number of rows in the data frame - This represents the number of games Mexico played in, as either team 1 or team 2
print("Number of matches " + str(df1.shape[0]))

# print the first few rows of the data frame
df1.head()

Number of matches 4


,index,num,date,time,score1,score2,score1i,score2i,group,city,timezone,team1.name,team1.code,team2.name,team2.code,stadium.key,stadium.name,score1et,score2et,score1p,score2p,knockout
0,10,11,2018-06-17,18:00,0,1,0,1,Group F,Moscow,UTC+3,Germany,GER,Mexico,MEX,luzhniki,Luzhniki Stadium,None,None,None,None,NaN
1,27,28,2018-06-23,18:00,1,2,0,1,Group F,Rostov-on-Don,UTC+3,South Korea,KOR,Mexico,MEX,rostov,Rostov Arena,None,None,None,None,NaN
2,43,44,2018-06-27,19:00,0,3,0,0,Group F,Ekaterinburg,UTC+5,Mexico,MEX,Sweden,SWE,ekaterinburg,Ekaterinburg Arena,None,None,None,None,NaN
3,52,53,2018-07-02,18:00,2,0,0,0,None,Samara,UTC+4,Brazil,BRA,Mexico,MEX,samara,Samara Arena,None,None,None,None,1.0


In [72]:
# From the world cup match data table, select only those rows that were tied games from the database and save these rows as a data frame
# HINT: By tie, this means that score1 is the same as score2
df1 = pd.read_sql_query('SELECT * FROM matches WHERE score1 = score2', con)

# count the number of rows in the data frame
print("Number of ties " + str(df1.shape[0]))

# print the first few rows of the data frame
df1.head()

Number of ties 14


,index,num,date,time,score1,score2,score1i,score2i,group,city,timezone,team1.name,team1.code,team2.name,team2.code,stadium.key,stadium.name,score1et,score2et,score1p,score2p,knockout
0,2,3,2018-06-15,21:00,3,3,2,1,Group B,Sochi,UTC+3,Portugal,POR,Spain,ESP,fisht,Fisht Stadium,NaN,NaN,NaN,NaN,NaN
1,6,7,2018-06-16,16:00,1,1,1,1,Group D,Moscow,UTC+3,Argentina,ARG,Iceland,ISL,spartak,Spartak Stadium,NaN,NaN,NaN,NaN,NaN
2,8,9,2018-06-17,21:00,1,1,1,0,Group E,Rostov-on-Don,UTC+3,Brazil,BRA,Switzerland,SUI,rostov,Rostov Arena,NaN,NaN,NaN,NaN,NaN
3,21,22,2018-06-21,16:00,1,1,1,1,Group C,Samara,UTC+4,Denmark,DEN,Australia,AUS,samara,Samara Arena,NaN,NaN,NaN,NaN,NaN
4,31,32,2018-06-24,20:00,2,2,1,1,Group H,Ekaterinburg,UTC+5,Japan,JPN,Senegal,SEN,ekaterinburg,Ekaterinburg Arena,NaN,NaN,NaN,NaN,NaN


In [73]:
con.close()